In [5]:
import pandas as pd
import polars as pl
import time

In [9]:
def timer(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        func(*args, **kwargs)
        end = time.perf_counter()
        return f"{func.__name__} took {end - start} seconds"
    return wrapper

In [10]:
def read_parquet_pandas():
    return pd.read_parquet('tpch-main/parquet/lineitem')

def read_parquet_polars():
    return pl.read_parquet('tpch-main/parquet/lineitem/*')

# timer(read_parquet_pandas)()
timer(read_parquet_polars)()

# import inspect

# print(inspect.getsource(read_parquet_polars))

'read_parquet_polars took 1.5037989160045981 seconds'

In [109]:
# Query 1
# SELECT
#     l_returnflag,
#     l_linestatus,
#     sum(l_quantity) as sum_qty,
#     sum(l_extendedprice) as sum_base_price,
#     sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
#     sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
#     avg(l_quantity) as avg_qty,
#     avg(l_extendedprice) as avg_price,
#     avg(l_discount) as avg_disc,
#     count(*) as count_order
# FROM
#     lineitem
# WHERE
#     l_shipdate <= date '1998-12-01' - interval '90' day
# GROUP BY
#     l_returnflag,
#     l_linestatus
# ORDER BY
#     l_returnflag,
#     l_linestatus;
import datetime as dt
def query1_pandas():
    df = pd.read_parquet('tpch-main/parquet/lineitem')
    df = df[df['l_shipdate'] <= dt.date(1998, 12, 1) - dt.timedelta(days=90)]
    df = df.groupby(['l_returnflag', 'l_linestatus']).agg(
        sum_qty=('l_quantity', 'sum'),
        sum_base_price=('l_extendedprice', 'sum'),
        sum_disc_price=('l_extendedprice', lambda x: (x * (1 - df['l_discount'])).sum()),
        sum_charge=('l_extendedprice', lambda x: (x * (1 - df['l_discount']) * (1 + df['l_tax'])).sum()),
        avg_qty=('l_quantity', 'mean'),
        avg_price=('l_extendedprice', 'mean'),
        avg_disc=('l_discount', 'mean'),
        count_order=('l_orderkey', 'count')
    )
    df = df.sort_values(['l_returnflag', 'l_linestatus'])
    return df.reset_index()



timer(query1_pandas)()


query1_pandas took 5.240994082996622 seconds


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,37734107.0,5.658655e+10,5.375826e+10,5.590907e+10,25.522006,38273.129735,0.049985,1478493
1,N,F,991417.0,1.487505e+09,1.413082e+09,1.469649e+09,25.516472,38284.467761,0.050093,38854
2,N,O,74476023.0,1.117017e+11,1.061182e+11,1.103670e+11,25.502230,38249.123838,0.049997,2920373
3,R,F,37719753.0,5.656804e+10,5.374129e+10,5.588962e+10,25.505794,38250.854626,0.050009,1478870


In [103]:
df = pl.read_parquet('tpch-main/parquet/lineitem/*')


In [108]:
# Query 1
# SELECT
#     l_returnflag,
#     l_linestatus,
#     sum(l_quantity) as sum_qty,
#     sum(l_extendedprice) as sum_base_price,
#     sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
#     sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
#     avg(l_quantity) as avg_qty,
#     avg(l_extendedprice) as avg_price,
#     avg(l_discount) as avg_disc,
#     count(*) as count_order
# FROM
#     lineitem
# WHERE
#     l_shipdate <= date '1998-12-01' - interval '90' day
# GROUP BY
#     l_returnflag,
#     l_linestatus
# ORDER BY
#     l_returnflag,
#     l_linestatus;
def query1_polars():
    return (
        df.filter(pl.col('l_shipdate') <= pl.date(1998, 12, 1) - pl.duration(days=90))
        .group_by(['l_returnflag', 'l_linestatus']).agg(
            sum_qty=pl.col('l_quantity').sum(),
            sum_base_price=pl.col('l_extendedprice').sum(),
            sum_disc_price=(pl.col('l_extendedprice') * (1 - pl.col('l_discount'))).sum(),
            sum_charge=(pl.col('l_extendedprice') * (1 - pl.col('l_discount') * (1 + pl.col('l_tax')))).sum(),
            avg_qty=pl.col('l_quantity').mean(),
            avg_price=pl.col('l_extendedprice').mean(),
            avg_disc=pl.col('l_discount').mean(),
            count_order=pl.col('l_orderkey').count()
        )
        .sort(['l_returnflag', 'l_linestatus'])
        )

timer(query1_polars)()


query1_polars took 0.687089333019685 seconds


l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
str,str,f64,f64,f64,f64,f64,f64,f64,u32
"""A""","""F""",3.7734107e7,5.6587e10,5.3758e10,5.3645e10,25.522006,38273.129735,0.049985,1478493
"""N""","""F""",991417.0,1.4875e9,1.4131e9,1.4101e9,25.516472,38284.467761,0.050093,38854
"""N""","""O""",7.4476023e7,1.1170e11,1.0612e11,1.0589e11,25.50223,38249.123838,0.049997,2920373
"""R""","""F""",3.7719753e7,5.6568e10,5.3741e10,5.3628e10,25.505794,38250.854626,0.050009,1478870


In [102]:

def read_titanic_pandas():
    titanic = pd.read_csv("data/titanic.csv")
    above_35 = titanic[titanic["Age"] > 35]
    return above_35.head()

def read_titanic_polars():
    return (
        pl.read_csv("data/titanic.csv")
        .filter(pl.col("Age") > 35)
        .head()
        )

timer(read_titanic_pandas)()
timer(read_titanic_polars)()

read_titanic_pandas took 0.002811084035784006 seconds
read_titanic_polars took 0.0010386669891886413 seconds


PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
7,0,1,"""McCarthy, Mr. …","""male""",54.0,0,0,"""17463""",51.8625,"""E46""","""S"""
12,1,1,"""Bonnell, Miss.…","""female""",58.0,0,0,"""113783""",26.55,"""C103""","""S"""
14,0,3,"""Andersson, Mr.…","""male""",39.0,1,5,"""347082""",31.275,null,"""S"""
16,1,2,"""Hewlett, Mrs. …","""female""",55.0,0,0,"""248706""",16.0,null,"""S"""


In [11]:
titanic = pd.read_csv("data/titanic.csv")
def filter_titanic_pandas():
    above_35 = titanic[titanic["Age"] > 35]
    return above_35

df = pl.read_csv("data/titanic.csv")
def filter_titanic_polars():
    return (
        df
        .filter(pl.col("Age") > 35)
        )

timer(read_titanic_pandas)()
timer(read_titanic_polars)()

NameError: name 'read_titanic_pandas' is not defined

In [3]:
import inspect

def query_1_pandas():
    df = pd.read_csv("data/titanic.csv")
    return df

# Get the full source code of the function
full_source = inspect.getsource(query_1_pandas)

# Split the source code into lines
source_lines = full_source.split('\n')

# Extract just the body of the function (excluding the signature)
body_lines = source_lines[1:-2]  # Exclude the first and last line which contain def and the closing parenthesis respectively

# Join the lines back together
body_source = '\n'.join(body_lines)

print(body_source)


    df = pd.read_csv("data/titanic.csv")
